In [1]:
import os
import logging
import numpy as np
import pandas as pd

import riskflow_jupyter

In [2]:
w=riskflow_jupyter.Workbench(
    path='U:\\CVA_JSON', 
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'},
    file_transform={
    'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
    'MarketData.dat': 'MarketData.json'})



output widget for  mtm  - TODO
output widget for  cashflows  - TODO


In [ ]:
w.context.current_cfg.deals['Deals']['Children'][0]['Children'][0]['Instrument'].field

In [ ]:
%debug

In [ ]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
cx = riskflow_jupyter.rf.Context(
    path_transform=path_transform,
    file_transform={
        'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
        'MarketData.dat': 'MarketData.json'
    })

In [ ]:
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Russellstone_Treasury_ISDA.json'))
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_BNP_Paribas__Paris__ISDA.json'))
cx.load_json(os.path.join(path, 'InputAAJ_CrB_Citibank_NA_NY_ISDA.json'))
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Natixis_ISDA.json'))



In [ ]:
import pandas as pd
a=pd.NaT
if a:
    print(sad)

In [ ]:
calc, output = cx.run_job()